![idea card](images/card_lunar_new_year_rat.svg)

## Installation and Overview

At this early stage in the project, please install from GitHub. Either clone the repository or use the ```pip``` command below. 

```pip install git+https://github.com/rn123/Calendrical-Tools#egg=Calendrical-Tools```
 
Once the package is installed, the three lines below are the minimal require to **generate and print** a calendar formated as a stacked list of ISO week numbers and weeks. 

```
from calendrical_tools import candybar
cal = candybar.TextCandyBar(2020)
cal.prcandybar()```

Besides the ```Calendrical Tools``` package, this project uses the fundemntal work of Reingold & Dershowitz, Calendrical Calculations. In fact, one of the main motivations of this project is part of the close reading of Reingold & Dershowitz -- being able to reproduce the calculations and figures in their work and to develop new diagrams and illustrations to explore the topics. 

Reingold & Dershowitz have a Common Lisp implementation (Calendrica 3.0) which was ported to Python 3, [```pycalcal```](https://github.com/espinielli/pycalcal).

- Reingold, Edward M. Calendrical Calculations: The Ultimate Edition. 4 edition. Cambridge ; New York: Cambridge University Press, 2018.

In [ ]:
from pycalcal import pycalcal as pcc
from calendrical_tools import candybar
from calendrical_tools.generate_astrolabe import *

# Use the jinja package to separate the formatting of the calendars and diagrams 
# (e.g. LaTex and SVG formats) from the calendrical computations.
from jinja2 import Template

import math
from collections import namedtuple
from IPython.core.display import SVG, Image

In [ ]:
# Hack: pip commands used in development and testing.
!pip uninstall calendrical_tools -y
!pip install git+https://github.com/rn123/Calendrical-Tools#egg=Calendrical-Tools

## Text CandyBar

A plain text candybar is the default output. When the code is first run for a new year, a file containing lunar data will be generate which cound take a minute.

In [ ]:
year = 2020
cal = candybar.TextCandyBar(year=year, weeks_before=1)

Generate a calendar and display it as plain text. The current default displays the Gregorian calendar for year, with all of the weeks of the year stacked one above the other. Also, the calendar prints out the ISO week number on the left. New moons are displayed as ```NM```.

<pre>
52	23 24 25 NM 27 28 29
 1	30 31  1  2  3  4  5
 2	 6  7  8  9 10 11 12
 3	13 14 15 16 17 18 19
 4	20 21 22 23 NM 25 26
 5	27 28 29 30 31  1  2
 6	 3  4  5  6  7  8  9
 7	10 11 12 13 14 15 16
 8	17 18 19 20 21 22 NM
 9	24 25 26 27 28 29  1
</pre>

**TODO:**
- Need more consistent interface to the different formatting classes.
- Need options to generate calendrical data for larger and shorter time periods.

In [ ]:
%%capture capture --no-stderr
# Hack, but useful, to grab textual output from command.

cal.prcandybar()

In [ ]:
print(capture.stdout)

In [ ]:
# Current default generates four calendars, debugging statement below to check 
# that four calendars were generated and each consists of the same number of weeks.
for cal_type in cal.weeks:
    print(len(cal.weeks[cal_type]), cal_type)

## SVG CandyBar

In [ ]:
cal = candybar.SvgCandyBar()

In [ ]:
for cal_type in cal.weeks:
    print(len(cal.weeks[cal_type]), cal_type)

Styling the output can be done by updating the style parameters in the candybar object. The 

In [ ]:
cal_color = {
    "iso": "grey;",
    "dim": "lightblue;",
    "highlight": "green;",
    "highlight_bold": "red;",
    "background": "yellow;"
}

cal_color = {
    "iso": "#cc232a; opacity: 0.5;",
    "dim": "#cc232a;",
    "highlight": "#f5ac27;",
    "highlight_bold": "#cc232a;",
    "background": "#a3262a;"
}

cal.bar_heading = ""
cal.cal_color = cal_color
cal.prcandybar()

In [ ]:
SVG(cal.svg)

## Astrolabe Diagram

The ```Astrolabe``` class computes a diagram providing a local view of the sky. The main parameter is the ```latitude``` of the location. To facilitate travelers, early astrolabes were constructed with with exchangeble 
plates. Quoting James Morrison:

> The earliest astrolabes, which were deeply influenced by Greek tradition,
    included plates for the latitudes of the *climates.* The climates of the world
    were defined by Ptolemy to be the latitudes where the lenght of the longest
    day of the year varied by one-half hour. Ptolemy calculated the latitude
    corresponding to a 15-minute difference in the length of the longest day
    (using a value of 23 degrees 51 minutes 20 seconds for the obliquity of
    the ecliptic) for 39 latitudes, which covered the Earth from the equator
    to the North Pole. The ones called the classic *climata* were for the
    half-hour differences in the longest day covering the then populated world."""

In [ ]:
plate_parameters = {"Hawaiian Islands": 21.3069}
astrolabe = Astrolabe(plate_parameters=plate_parameters)
plate = astrolabe.plates["Hawaiian Islands"]

The current version inclues a short animation, showing the motion of the ecliptic across the local sky.

In [ ]:
animation_parameters = {"from": "0", "to": "233", "begin": "0s", "dur": "5s"}

with open("../calendrical_tools/astrolabe_template.svg") as fp:
    template_text = fp.read()

In [ ]:
# Use Inkscape extensions to svg to place different parts of astrolabe into their own layer.
inkscape_attributes = {
    identifier: 'inkscape:label="{}" inkscape:groupmode="layer"'.format(identifier)
    for identifier in identifiers
}

In [ ]:
ecliptic={
        "cx": astrolabe.xEclipticCenter,
        "cy": astrolabe.yEclipticCenter,
        "r": astrolabe.RadiusEcliptic,
        "width": 5,
    }

In [ ]:
outer_radius = ecliptic["r"] 
inner_radius = ecliptic["r"] - ecliptic["width"]

top_middle_outer =    {"x":(ecliptic["cx"]), "y":(ecliptic["cy"] + outer_radius)}
bottom_middle_outer = {"x":(ecliptic["cx"]), "y":(ecliptic["cy"] - outer_radius)}

top_middle_inner =    {"x":(ecliptic["cx"]), "y":(ecliptic["cy"] + inner_radius)}
bottom_middle_inner = {"x":(ecliptic["cx"]), "y":(ecliptic["cy"] - inner_radius)}

In [ ]:
template = Template(template_text)
astrolabe_svg = template.render(
    place_name=plate["location"],
    latitude=plate["latitude"],
    RCapricorn=astrolabe.RadiusCapricorn,
    REquator=astrolabe.RadiusEquator,
    RCancer=astrolabe.RadiusCancer,
    horiz=plate["horizon"],
    almucantor_coords=plate["almucantars"],
    almucantar_center=plate["almucantar_center"],
    azimuth_coords=plate["azimuths"],
    prime_vertical=plate["prime_vertical"],
    ticks=astrolabe.ticks,
    ecliptic=ecliptic,
    top_middle_outer=top_middle_outer,
    bottom_middle_outer=bottom_middle_outer,
    outer_radius=outer_radius,
    inner_radius=inner_radius,
    top_middle_inner=top_middle_inner,
    bottom_middle_inner=bottom_middle_inner,
    ecliptic_center=astrolabe.RadiusEquator * math.tan(astrolabe._obliquityRadiansArgument),
    stroke_color=cal_color["highlight"],
    background_color=cal_color["background"],
    inkscape=inkscape_attributes,
    animation=animation_parameters,
    )

In [ ]:
SVG(data=astrolabe_svg)

## Dividing the Ecliptic

In [ ]:
ecliptic_center = astrolabe.RadiusEquator * math.tan(astrolabe._obliquityRadiansArgument)
ecliptic_pole = astrolabe.RadiusEquator * math.tan(astrolabe._obliquityRadiansArgument/2)

In [ ]:
print(ecliptic_center, ecliptic_pole)

In [ ]:
ecliptic_division_template = """
<svg viewbox="0 0 200 200" 
     width="450" height="450" 
     xmlns="http://www.w3.org/2000/svg" 
     xmlns:inkscape="http://www.inkscape.org/namespaces/inkscape" 
     xmlns:xlink="http://www.w3.org/1999/xlink">
<defs>
    <style type="text/css">
        #tropicCircles {
          fill: none;
          stroke: {{ stroke_color }}
          stroke-width: 1;
          stroke-opacity: 0.5;
        }
        #axisFigure {
          stroke: {{ stroke_color }}
          stroke-width: 1;
          stroke-opacity: 0.5;
        }
    </style>
</defs>

<g id="test" transform="translate(100,100)">
    <title>Ecliptic Division</title>
    <g id="tropicCircles">
        <circle cx="0" cy="0" r="{{ RCapricorn }}"/>
        <circle cx="0" cy="0" r="{{ REquator }}" style="stroke: black;"/>
        <circle cx="0" cy="0" r="{{ RCancer }}"/>
    </g>
    
    <g id="axisFigure">
        <line x1="0" y1="{{ -RCapricorn }}" x2="0" y2="{{ RCapricorn }}"/>
        <line x1="{{ -RCapricorn }}" y1="0" x2="{{ RCapricorn }}" y2="0"/>
    </g>
    
    <path style="stroke:gold; fill:none" d="
        M0 {{ ecliptic.cy + ecliptic.r }}
        A{{ ecliptic.r }} {{ ecliptic.r }} 0 0 1 {{ ecliptic.cx }} {{ ecliptic.cy - ecliptic.r }}
        A{{ ecliptic.r }} {{ ecliptic.r }} 0 0 1 {{ ecliptic.cx }} {{ ecliptic.cy + ecliptic.r }}z"/>
    
    <g>
        <line x1="-5" y1="{{ ecliptic_center }}" x2="5" y2="{{ ecliptic_center }}" style="stroke: black;"/>
        <line x1="0" y1="{{ ecliptic_center - 5 }}" x2="0" y2="{{ ecliptic_center + 5 }}" style="stroke: black;"/>
    </g>
    
    <g>
        <title>Divide Equator</title>
        <line id="tickMark" x1="0" y1="{{ REquator - 2 }}" x2="0" y2="{{ REquator + 2 }}" style="stroke: black; stroke-width: 1;"/>
        {% for angle in angles %}
            <use xlink:href="#tickMark" transform="rotate({{ angle }})"/>
        {%- endfor %}
    </g>
</g>
</svg>
"""

In [ ]:
angles = list(range(0, 361, 30))

template = Template(ecliptic_division_template)
ecliptic_division_svg = template.render(
    RCapricorn=astrolabe.RadiusCapricorn,
    REquator=astrolabe.RadiusEquator,
    RCancer=astrolabe.RadiusCancer,
    ecliptic_center=ecliptic_center,
    ecliptic=ecliptic,
    stroke_color="black;",
    angles=angles,
)

SVG(ecliptic_division_svg)

In [ ]:
for angle in angles:
    angleRadians = math.radians(angle)
    print(math.tan(angleRadians))

In [ ]:
with open('ecliptic_division.svg', 'w') as fp:
    fp.write(ecliptic_division_svg)

In [ ]:
test = """
<svg viewbox="0 0 300 300" 
     width="500" height="500" 
     xmlns="http://www.w3.org/2000/svg" 
     xmlns:inkscape="http://www.inkscape.org/namespaces/inkscape" 
     xmlns:xlink="http://www.w3.org/1999/xlink">
    <defs>
        <style type="text/css">
            #tropicCircles {
              fill: none;
              stroke: black;
              stroke-width: 1;
              stroke-opacity: 0.5;
            }
            #eclipticCircle {
              fill: none;
              stroke: gold;
              stroke-width: 2;
              stroke-opacity: 1;
            }
            #axisFigure {
              stroke: black;
              stroke-width: 1;
              stroke-opacity: 0.5;
            }
            #division {
              stroke: black;
              stroke-width: 1;
              stroke-opacity: 0.5;
            }
            #divisionBold {
              stroke: red;
              stroke-width: 1;
              stroke-opacity: 1;
            }
            #eclipticPole, #eclipticCenter {
              stroke: black;
              stroke-width: 1;
              stroke-opacity: 0.5;
            }
        </style>
    </defs>
        
     <g id="test" transform="translate(150,150)">
        <path id="tropicCircles" d="
            M0 {{ RCapricorn}}
            A{{ RCapricorn }} {{ RCapricorn }} 0 0 1 {{ 0 }} {{ -RCapricorn }}
            A{{ RCapricorn }} {{ RCapricorn }} 0 0 1 {{ 0 }} {{ RCapricorn }}z"/>
            
        <path id="tropicCircles" d="
            M0 {{ REquator}}
            A{{ REquator }} {{ REquator }} 0 0 1 {{ 0 }} {{ -REquator }}
            A{{ REquator }} {{ REquator }} 0 0 1 {{ 0 }} {{ REquator }}z"/>
            
        <path id="tropicCircles" d="
            M0 {{ RCancer}}
            A{{ RCancer }} {{ RCancer }} 0 0 1 {{ 0 }} {{ -RCancer }}
            A{{ RCancer }} {{ RCancer }} 0 0 1 {{ 0 }} {{ RCancer }}z"/>
            
        <path id="eclipticCircle" d="
            M0 {{ ecliptic.cy + ecliptic.r }}
            A{{ ecliptic.r }} {{ ecliptic.r }} 0 0 1 {{ ecliptic.cx }} {{ ecliptic.cy - ecliptic.r }}
            A{{ ecliptic.r }} {{ ecliptic.r }} 0 0 1 {{ ecliptic.cx }} {{ ecliptic.cy + ecliptic.r }}z"/>   
        
        {% for division in equator_division %}
            <line id="division" x1="0" y1="0" x2="{{ (REquator + 40) * division.x2 }}" 
                  y2="{{ (REquator + 40) * division.y2 }}"/>
        {%- endfor %}
        
        <line id="divisionBold" x1="0" y1="0" x2="{{ (REquator + 40) * sample_division.x2 }}" 
              y2="{{ (REquator + 40) * sample_division.y2 }}"/>
        
        <line id="eclipticCenter" x1="-5" y1="{{ ecliptic_center }}" x2="5" y2="{{ ecliptic_center }}"/>
        <line id="eclipticCenter" x1="0" y1="{{ ecliptic_center - 5 }}" x2="0" y2="{{ ecliptic_center + 5 }}"/>
  
        <line id="eclipticPole" x1="-5" y1="{{ ecliptic_pole }}" x2="5" y2="{{ ecliptic_pole }}"/>
        <line id="eclipticPole" x1="0" y1="{{ ecliptic_pole - 5 }}" x2="0" y2="{{ ecliptic_pole + 5 }}"/>

        {% for division in equator_division %}
            <line id="division" x1="{{ ecliptic_pole.cx }}" 
                                y1=" {{ ecliptic_pole.cy }}" 
                                x2="{{ intersections.x2 }}" 
                                y2="{{ intersections.y2 }}"/>
        {%- endfor %}
    </g>
     
</svg>
"""

In [ ]:
# !pip install svgwrite
# !pip install svgpathtools

In [ ]:
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc

In [ ]:
from svgpathtools import svg2paths, wsvg, disvg

In [ ]:
paths, attributes = svg2paths('ecliptic_division.svg')

In [ ]:
redpath = paths[0]
redpath_attribs = attributes[0]

In [ ]:
intersections = []
for path in paths:
    if path != redpath:
        for (T1, seg1, t1), (T2, seg2, t2) in redpath.intersect(path):
            p = redpath.point(T1)
            intersections.append(p)

intersections = [i for i in intersections if i is not None]
intersections = list(set([(p.real, p.imag) for p in intersections]))

match_list = []
while len(intersections) > 0:
    p = intersections.pop()
    l = [p]
    for m, q in enumerate(intersections):
        if math.isclose(p[0], q[0], abs_tol=0.01) and math.isclose(p[1], q[1], abs_tol=0.01):
            l.append(q)
            intersections.pop(m)
    match_list.append(l)

match_list = [{"x2":m[0][0], "y2":m[0][1]} for m in match_list]

In [ ]:
angles = list(range(0, 361, 30))
sample_angle = 30

template = Template(test)
ecliptic_division_svg = template.render(
    RCapricorn=astrolabe.RadiusCapricorn,
    REquator=astrolabe.RadiusEquator,
    RCancer=astrolabe.RadiusCancer,
    ecliptic=ecliptic,
    stroke_color="black;",
    ecliptic_center=ecliptic_center,
    ecliptic_pole=ecliptic_pole,
    equator_division=[{"x2":math.cos(math.radians(angle)), 
                       "y2":math.sin(math.radians(angle)), 
                       "angle":angle} for angle in angles],
    sample_division={"x2":math.cos(math.radians(sample_angle)), 
                     "y2":math.sin(math.radians(sample_angle)), 
                     "angle":sample_angle},
    intersections=match_list
)

# with open('ecliptic_division.svg', 'w') as fp:
#     fp.write(ecliptic_division_svg)
    
SVG(ecliptic_division_svg)

In [ ]:
disvg([redpath, seg2])

In [ ]:
disvg(paths, filename='output_intersections.svg', attributes=attributes,
      nodes = intersections, node_radii = [5]*len(intersections))

## Sun, moon, and stars

Motion of moon and sun is computed along the ecliptic. In order to plot the position of the moon and sun on the plane of the astrolabe (actually onto the rete), first convert the ecliptic longitude and latitude to equatorial coordinates, and then use the stereographic projection.

The ```CandyBar``` object contains the computed new moons of the time period. This is a list of the 

In [ ]:
lunar_new_year = cal.new_moons[737448]
lunar_new_year

$\sin\delta = \sin\beta\cos\epsilon + \cos\beta \sin\epsilon \sin\lambda$

Since $\beta = 0$ for the sun, the formula simplifies to:

$\sin\delta = \sin\epsilon \sin\lambda$ 

In [ ]:
lunar_new_year.moon_sidereal_longitude

In [ ]:
sin_declination = (math.sin(astrolabe._obliquityRadians) 
                    * math.sin(math.radians(lunar_new_year.moon_sidereal_longitude)))
declination_radians = pcc.asin(sin_declination)
declination = pcc.degrees(declination_radians)
declination_argument = math.radians((90 - declination) / 2.0)
r = astrolabe.RadiusEquator * math.tan(declination_argument)

In [ ]:
lunar_latitude = pcc.degrees(pcc.lunar_latitude(lunar_new_year.moon_fixed_day))
lunar_latitude_radians = math.radians(lunar_latitude)

tan_right_ascension = (
    (math.sin(lunar_latitude_radians) * math.cos(astrolabe._obliquityRadians) 
        - math.tan(declination_radians) * math.sin(astrolabe._obliquityRadians)) / 
     math.cos(lunar_latitude_radians)
)

alpha_radians = pcc.atan( tan_right_ascension )
alpha = pcc.degrees(alpha_radians)

In [ ]:
print(r, alpha)

## 鼠年大吉

In [ ]:
with open("../docs/images/rat.svg") as fp:
    rat_svg = fp.read()

In [ ]:
rat_template = """
<svg viewbox="0 0 300 300" 
     width="300" height="300" 
     xmlns="http://www.w3.org/2000/svg" 
     xmlns:inkscape="http://www.inkscape.org/namespaces/inkscape" 
     xmlns:xlink="http://www.w3.org/1999/xlink">
<defs>
    <style type="text/css">
        #983: { fill: red;
        } 
    </style>
</defs>
<g id="rat">
    <title>Year of the Rat </title>
        {{ rat }}
</g>
</svg>
"""

In [ ]:
template = Template(rat_template)
svg = template.render(rat=rat_svg, background=cal_color["background"])

In [ ]:
SVG(svg)

## Concept Card for Project

In [ ]:
card_template = """
<svg viewbox="0 0 1280 640" 
     width="1280" height="640" 
     xmlns="http://www.w3.org/2000/svg" 
     xmlns:inkscape="http://www.inkscape.org/namespaces/inkscape" 
     xmlns:xlink="http://www.w3.org/1999/xlink">
     
<g id="card">
    
    <defs>
        <g transform="scale(1.3)">
            <clipPath id="hole">
               <path d="M75 75 L 1205 75 L 1205 565 L 75 565Z" 
                     style="stroke: {{ cal_color.highlight }} 
                     fill: {{ cal_color.background }}"/>
            </clipPath>
        </g>
    </defs>
    
    <g>
        <rect x="75px" y="75px" width="1130" height="490" 
              style="stroke: {{ cal_color.highlight }} fill: {{ cal_color.background }}"/>
    </g>

    <g transform="translate(560, 138)">
        <title>New Year Greeting</title>
        <text x="65" y="40" writing-mode="tb" 
              style="font-size:60; font-family: Courier Arial, Helvetica, sans-serif; fill:{{ cal_color.highlight }} fill-opacity:1.0;">鼠年大吉
            <tspan x="0" y="40" writing-mode="tb-rl" style="font-size: 60;">恭喜發財</tspan>
        </text>
        <line x1="-30" y1="40" x2="-30" y2="320" style="stroke: {{ cal_color.highlight }} stroke-opacity: 0.2; stroke-width:3" />
        <line x1="35"  y1="40" x2="35"  y2="320" style="stroke: {{ cal_color.highlight }} stroke-opacity: 0.2; stroke-width:3" />
        <line x1="100" y1="40" x2="100" y2="320" style="stroke: {{ cal_color.highlight }} stroke-opacity: 0.2; stroke-width:3" />
    </g>
    
    <g>
        <text x="95" y="125"
              style="font-size:30; fill: {{ cal_color.highlight }};fill-opacity:1.0;">Calendrical Tools 2020</text>
        <g transform="translate(1110, 435) scale(0.55)">
            <title>Rat</title>
            {{ rat }}
        </g>
    </g>
    
    <g transform="translate(690, 70)">
        <g transform="scale(2)">
            {{ astrolabe}}
        </g>
    </g>
    
    <g style="clip-path: url(#hole);">
        <rect x="75" y="155" width="400" height="430" style="stroke: {{ cal_color.highlight }} fill:none;"/>
        <rect x="65" y="165" width="400" height="430" style="stroke: {{ cal_color.highlight }} fill:none;"/>
        <g transform="translate(75, 155)">
            {{ candybar }}
        </g>
    </g>
    
</g></svg>
""" 

In [ ]:
template = Template(card_template)
astrolabe_svg = astrolabe_svg.replace('21.3069', "21° 18' 25''")
card_svg = template.render(candybar=cal.svg, astrolabe=astrolabe_svg, 
                           rat=rat_svg, cal_color=cal_color)
SVG(card_svg)

In [ ]:
with open('card.svg', 'w') as fp:
    fp.write(card_svg)

**TODO:**
- The current ``png`` image displays the vertical strings of Chinese character incorrectl. May be an issues with ```cairosvg```.
- Not an issue, ```writing-mode``` not supported.

In [ ]:
try:
    import cairosvg
except Exception as ex:
    print('Exception {}. Install cairosvg: '.format(ex))
    !pip install cairosvg

In [ ]:
def png_from_svg(filename="card.svg"):
    with open(filename) as fp:
        card_svg = fp.read()
    
    return cairosvg.svg2png(card_svg)

In [ ]:
png = png_from_svg()
Image(png)